# Steamflow Eval :: DScore Analysis

<img src='../../../doc/assets/Eval_Analysis.svg' width=600>

:::{note}

_This notebook adapted from originals by Timothy Hodson and Rich Signell. See that upstream work at:_
* https://github.com/thodson-usgs/dscore
* https://github.com/USGS-python/hytest-evaluation-workflows/

:::


## Essential Benchmark Components
This benchmark notebook will present a workflow which follows a canonical model for Essential Benchmark Components: 
1) A set of predictions and matching observations; 
2) The domain (e.g. space or time) over which to benchmark;
3) A set of statistical metrics with which to benchmark. 

Let's get started.

## Step 0: Load libraries and configure Python computing environment

In [ ]:
import pandas as pd
import logging
import os
# Needed when boto3 >= 1.36.0 or the rechunking process will fail
# This needs to be set before the boto3 library gets loaded
# See: https://github.com/aws/aws-cli/issues/9214#issuecomment-2606619168
os.environ['AWS_REQUEST_CHECKSUM_CALCULATION'] = 'when_required'
os.environ['AWS_RESPONSE_CHECKSUM_VALIDATION'] = 'when_required'

## Step 1: Load Data

<img src='../../../doc/assets/Eval_Analysis_Data.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  <span style="color:red; font-size:large"><<--You are here</span>
2) The domain over which to benchmark 
3) A set of statistical metrics with which to benchmark. 

Finding and loading data is made easier for this particular workflow (the _streamflow_ variable), in that most of it has been 
pre-processed and stored in a cloud-friendly format.  That process is described in the [data preparation](01_Data_Prep.ipynb)
notebook. We will proceed here using the already-prepared data for _streamflow_, which is included in the HyTEST **intake catalog**.  

:::{margin}
Learn more about `intake` [here](../../../dataset_catalog/README.md)
:::

In [ ]:
import intake
cat = intake.open_catalog(r'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml')
print("Available datasets: \n", "\n".join(cat.keys()))

The above list represents the processed datasets available for benchmarking.  If a dataset
you want is not in that list, 
you can load data directly from S3 or onprem. 
If you load data from a source other than this list, you can jump to Step 2: Restrict to a Domain

Note that the interesting datasets in the cataloged dataset above are duplicated: Some are `-onprem` , some are `-cloud`, and some are some are `-osn`. Same data, but the storage location and access protocol will be different. You 
will definitely want to use the correct copy of the data for your computing environment.  
* `onprem` : Direct access via the `caldera` filesystem from _denali_ or _tallgrass_
* `cloud` : Network access via S3 bucket, suitable for consumption on cloud-hosted jupyter servers. You could also access using any network-attached computer, but the amount of data will likely saturate your connection.  Use in the cloud (e.g. ESIP QHub)
* `osn` : Network access via OSN pod, which uses the S3 API, suitable for consumption on any jupyter server.

So... are you on-prem? 

In [ ]:
import platform
onprem = (platform.node() in ['denali', 'tallgrass'])
if onprem:
    print("Yes : -onprem")
    obs_data_src='nwis-streamflow-usgs-gages-onprem'
    mod_data_src='nwm21-streamflow-usgs-gages-onprem'
else:
    print("Not onprem; use '-osn' data source")
    obs_data_src='nwis-streamflow-usgs-gages-osn'
    mod_data_src='nwm21-streamflow-usgs-gages-osn'
print("Observed : ", obs_data_src)
print("Modeled  : ", mod_data_src)

In [ ]:
variable_of_interest = 'streamflow'
try:
    obs = cat[obs_data_src].to_dask()
    mod = cat[mod_data_src].to_dask()
except KeyError:
    print("Something wrong with dataset names.")
    raise

try:
    obs_data = obs[variable_of_interest]
    mod_data = mod[variable_of_interest].astype('float32')
except KeyError:
    print(f"{variable_of_interest} was not found in these data.")

obs_data.name = 'observed'
mod_data.name = 'predicted'    

## Step 2: Restrict to a Domain

<img src='../../../doc/assets/Eval_Analysis_Domain.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  
2) The domain over which to benchmark <span style="color:red; font-size:large"><<--You are here</span>
3) A set of statistical metrics with which to benchmark. 

Each benchmark domain is defined over specific bounds (typically space and/or time). 
Benchmark domain definitions can be defined within the notebook, or sourced from
elsewhere. For this example, we use the _Cobalt_ gages, avaliable for download on ScienceBase 
([Foks et al., 2022](https://doi.org/10.5066/P972P42Z)).  

This is essentially a list of 
stream guages in which we are interested, along with some  metadata about that gage (watershed, 
reach code, etc).  We will use this as a spatial selector to restrict the original data to only 
those gages found within this benchmarking domain.

In [ ]:
cobalt = pd.read_csv(
    'https://www.sciencebase.gov/catalog/file/get/6181ac65d34e9f2789e44897?f=__disk__22%2F1a%2Fd2%2F221ad2fe9d95de17731ad35d0fc6b417a4b53ee1',
    dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str, 'gagesII_class':str, 'aggecoregion': str}, 
    index_col='site_no'
    )
# Re-format the gage_id/site_no string value.  ex:   "1000000"  ==> "USGS-1000000"
cobalt.rename(index=lambda x: f'USGS-{x}', inplace=True)
print(f"{len(cobalt.index)} gages in this benchmark")
cobalt.head(2)

So now we have a domain dataset representing the stream gages (unique `site_no` values) identifying the locations making up the spatial domain of this benchmark. While we have good meta-data for these gages (lat/lon location, HUC8 code, etc), we really will only use the list of `site_no` values to select locations out of the raw data.

## Step 3: Define Metrics

<img src='../../../doc/assets/Eval_Analysis_Metrics.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  
2) The domain over which to benchmark 
3) A set of statistical metrics with which to benchmark. <span style="color:red; font-size:large"><<--You are here</span>

The code to calculate the various metrics has been standardized [here](../../Metrics_StdSuite_v1). 
You can use these or write your own.  To import and use these standard definitions, run this cell:

In [ ]:
%run ../../Metrics_DScore_Suite_v1.ipynb

Whether you use these functions or your own, we need to put all metric computation into a special all-encompasing 
benchmarking function--a single call which can be assigned to each gage in our domain list. This sort of arrangement 
is well-suited to parallelism with `dask`. 

If this is done well, the process will benefit enormously from task parallelism -- each gage can be given its own 
CPU to run on.  After all are done, the various results will be collected and assembled into a composite dataset. 

To achieve this, we need a single 'atomic' function that can execute independently. It will take the gage identifier 
as input and return a list of metrics.

In [ ]:
## Wrapper function -- this func will be called once per gage_id, each call on its own dask worker
def compute_benchmark(gage_id):
    try:
        ## obs_data and mod_data should be globals...
        obs = obs_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series()
        mod = mod_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series().resample('1D', offset='5h').mean() 
        
        # make sure the indices match
        obs.index = obs.index.to_period('D')
        mod.index = mod.index.to_period('D')

        # merge obs and predictions; drop NaNs.
        gage_df = pd.merge(obs, mod, left_index=True, right_index=True).dropna(how='any')
        
        obs_log = np.log(gage_df['observed'].clip(lower=0.01)) # clip to remove zeros and negative values
        sim_log = np.log(gage_df['predicted'].clip(lower=0.01))
        
        scores = pd.concat([
                pd.Series([ mse(obs_log, sim_log) ], index=["mse"], dtype='float64'),
                bias_distribution_sequence(obs_log, sim_log), 
                seasonal_mse(obs_log, sim_log),
                quantile_mse(obs_log, sim_log)
                ]
            )
        scores.name=gage_id
        return scores
    except Exception as e:#<-- this is an extremely broad way to catch exceptions.  We only do it this way to ensure 
            #    that a failure on one benchmark (for a single stream gage) will not halt the entire run. 
        logging.info("Benchmark failed for %s", gage_id)
        return None

Let's test to be sure this `compute_benchmark()` function will return data for a single gage

In [ ]:
compute_benchmark('USGS-01030350')

## Execute the Analysis 
We will be doing a lot of work in paralallel, using workers within a 'cluster'.  
The details of cluster configuration are handled for us by 'helper' notebooks, below. 
You can override their function by doing your own cluster configuration if you like. 

In [ ]:
# uncomment the lines below to read in your AWS credentials if you want to access data from a requester-pays bucket (-cloud)
# os.environ['AWS_PROFILE'] = 'default'
# %run ../../../environment_set_up/Help_AWS_Credentials.ipynb

In [ ]:
%run ../../../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
### Executes external 'helper to spin up a cluster of workers. 

We verified above that the `compute_benchmark` works on the "hosted" server (where this
notebook is being executed. As a sanity check before we give the cluster of workers a lot 
to do, let's verify that we can have a remote worker process a gage by submitting work
to one in isolation: 

In [ ]:
client.submit(compute_benchmark, 'USGS-01030350').result()

Now that we've got a benchmark function, and can prove that it works in remote workers 
within the cluster, we can dispatch a fleet of workers to process our data in parallel.
We will mak use of `dask` to do thisusing a dask '_bag_'.  

:::{margin}

Read more about task parallelism with Dask and how we are using dask bags [here](../../../essential_reading/Parallel_Dask.ipynb)

:::

In [ ]:
# Set up a dask bag with the contents beging a list of the cobalt gages
import dask.bag as db
bag = db.from_sequence( cobalt.index.tolist() ).map(compute_benchmark)
results = bag.compute() 

With that big task done, we don't need `dask` parallelism any more. Let's shut down the cluster:

In [ ]:
client.close(); del client
cluster.close(); del cluster

## Assemble the results
The `bag` now contains a collection of return values (one per call to `compute_benchmark()`).  We can massage that into a table/dataframe for easier processing: 

In [ ]:
results = [i for i in results if i is not None] # Drop entries where compute_benchmark failed
results_df = pd.concat(results, axis=1).T
results_df.index.name = 'site_no'
results_df

This dataframe/table can be saved to disk as a CSV. It will be used for visualizations [here](./03_Vizualization.ipynb).

In [ ]:
results_df.to_csv('DScore_streamflow_example.csv') ##<--- change this to a personalized filename